In [79]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

from statistics import mean
import ipyleaflet
import gpxpy
import pandas as pd
from ipyleaflet import basemaps, FullScreenControl, LayerGroup, Map, \
    MeasureControl, Polyline, Marker, CircleMarker, WidgetControl
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, \
    Label, Output, IntSlider, Layout, Image, link
import numpy as np

In [45]:
gpx_file_path = 'Gran_Fondo_Porto_SantElpidio_2022.gpx'

with open(gpx_file_path) as fh:
    gpxFile = gpxpy.parse(fh)

In [46]:
# create map
basic_map = ipyleaflet.Map(zoom = 1)
# basic_map = Map(zoom=4)

# display map
basic_map
# display(basic_map)

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [47]:
def plot_map(gpx):
    """
    Plot the GPS trace on a map
    """
    points = [p.point for p in gpx.get_points_data(distance_2d=True)]
    mean_lat = mean(p.latitude for p in points)
    mean_lng = mean(p.longitude for p in points)

    # create the map
    m = Map(center=(mean_lat, mean_lng), zoom=12, basemap=basemaps.Stamen.Terrain)

    # show trace
    line = Polyline(locations=[[[p.latitude, p.longitude] for p in points],],
                    color = "red", fill=False)
    m.add_layer(line)

    # add markers
    waypoints = [
        Marker(location=(point.latitude, point.longitude), title=point.name,
               popup=HTML(value=point.name), draggable=False)
        for point in gpx.waypoints
    ]
    waypoints_layer = LayerGroup(layers=waypoints)
    m.add_layer(waypoints_layer)
    
    # add a checkbox to show / hide waypoints
    waypoints_checkbox = Checkbox(value=True, description='Show Waypoints')
    
    def update_visible(change):
        for p in waypoints:
            p.visible = change['new']
    
    waypoints_checkbox.observe(update_visible, 'value')
    waypoint_control = WidgetControl(widget=waypoints_checkbox, position='bottomright')
    m.add_control(waypoint_control)
    
    # enable full screen mode
    m.add_control(FullScreenControl())
        
    # add measure control
    measure = MeasureControl(
        position='bottomleft',
        active_color = 'orange',
        primary_length_unit = 'kilometers'
    )
    m.add_control(measure)
        
    return m

In [49]:
plot_map(gpxFile)

Map(center=[43.14745246278689, 13.682265709631148], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [62]:
for j in range(89):
    print(gpxFile.get_points_data(distance_2d=True)[j])

PointData(point=GPXTrackPoint(43.2387, 13.75904, elevation=19.0, time=datetime.datetime(2022, 4, 18, 0, 0, tzinfo=SimpleTZ("Z"))), distance_from_start=0, track_no=0, segment_no=0, point_no=0)
PointData(point=GPXTrackPoint(43.23864, 13.75896, elevation=19.0, time=datetime.datetime(2022, 4, 18, 0, 0, 2, tzinfo=SimpleTZ("Z"))), distance_from_start=9.311409201481228, track_no=0, segment_no=0, point_no=1)
PointData(point=GPXTrackPoint(43.2386, 13.75896, elevation=20.0, time=datetime.datetime(2022, 4, 18, 0, 0, 2, tzinfo=SimpleTZ("Z"))), distance_from_start=13.764188833043598, track_no=0, segment_no=0, point_no=2)
PointData(point=GPXTrackPoint(43.23855, 13.75898, elevation=19.0, time=datetime.datetime(2022, 4, 18, 0, 0, 4, tzinfo=SimpleTZ("Z"))), distance_from_start=19.561668470737022, track_no=0, segment_no=0, point_no=3)
PointData(point=GPXTrackPoint(43.2385, 13.75907, elevation=19.0, time=datetime.datetime(2022, 4, 18, 0, 0, 5, tzinfo=SimpleTZ("Z"))), distance_from_start=28.74054562535595

PointData(point=GPXTrackPoint(43.19822, 13.78535, elevation=5.0, time=datetime.datetime(2022, 4, 18, 0, 17, 54, tzinfo=SimpleTZ("Z"))), distance_from_start=5967.399341021157, track_no=0, segment_no=0, point_no=74)
PointData(point=GPXTrackPoint(43.19756, 13.78567, elevation=5.0, time=datetime.datetime(2022, 4, 18, 0, 18, 8, tzinfo=SimpleTZ("Z"))), distance_from_start=6045.324513358796, track_no=0, segment_no=0, point_no=75)
PointData(point=GPXTrackPoint(43.19652, 13.7862, elevation=5.0, time=datetime.datetime(2022, 4, 18, 0, 18, 30, tzinfo=SimpleTZ("Z"))), distance_from_start=6168.828256211604, track_no=0, segment_no=0, point_no=76)
PointData(point=GPXTrackPoint(43.196, 13.78644, elevation=4.0, time=datetime.datetime(2022, 4, 18, 0, 18, 41, tzinfo=SimpleTZ("Z"))), distance_from_start=6229.903244173913, track_no=0, segment_no=0, point_no=77)
PointData(point=GPXTrackPoint(43.19578, 13.78657, elevation=4.0, time=datetime.datetime(2022, 4, 18, 0, 18, 46, tzinfo=SimpleTZ("Z"))), distance_fro

In [76]:
j = 0
gpxFile.get_points_data(distance_2d=True)[j].tracks().segments[j]
coords = pd.DataFrame([
    {'lat': p.latitude,
     'lon': p.longitude,
     'time': p.time} for p in setment.points])

AttributeError: 'PointData' object has no attribute 'tracks'

In [80]:
# Convert to a dataframe one point at a time.
points = []
for segment in gpxFile.tracks[0].segments:
    for p in segment.points:
        points.append({
            'time': p.time,
            'latitude': p.latitude,
            'longitude': p.longitude,
            'elevation': p.elevation,
        })
df = pd.DataFrame.from_records(points)

In [81]:
df.head()

,time,latitude,longitude,elevation
0,2022-04-18 00:00:00+00:00,43.23870,13.75904,19.0
1,2022-04-18 00:00:02+00:00,43.23864,13.75896,19.0
2,2022-04-18 00:00:02+00:00,43.23860,13.75896,20.0
3,2022-04-18 00:00:04+00:00,43.23855,13.75898,19.0
4,2022-04-18 00:00:05+00:00,43.23850,13.75907,19.0


In [74]:
# https://www.youtube.com/watch?v=jAKUPm-Cul0

'PointData(point=GPXTrackPoint(43.2387, 13.75904, elevation=19.0, time=datetime.datetime(2022, 4, 18, 0, 0, tzinfo=SimpleTZ("Z"))), distance_from_start=0, track_no=0, segment_no=0, point_no=0)'